In [ ]:
from ByBit import ByBitOptionData, recommend_option_position, ByBitAPI

In [ ]:
prod_url  = "https://api.bybit.com"
ticker_endpoint = "/v5/market/tickers"
btc_params = {'category': "option",
              'baseCoin': "BTC"}

daily_delta_limit: float = 0.10  # Daily Delta Deviation Limit
weekly_delta_limit: float = 0.10  # Weekly Delta Deviation Limit
monthly_delta_limit: float = 0.10  # Monthly Delta Deviation Limit
quarterly_delta_limit: float = 0.10  # Quarterly Delta Deviation Limit

In [ ]:
ByBit = ByBitOptionData(api_url = prod_url, api_endpoint = ticker_endpoint, api_parameters = btc_params)

ByBitAPI = ByBitAPI(default_quantity = 1.0,
    api_url = "https://api-demo.bybit.com",
    api_key = "6TzoOeOuyIDiJN9cOu",
    api_secret = "JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",
    baseCoin = "BTC",
    settleCoin= "USDT")

In [ ]:
await ByBit.fetch_ByBit_ticker_data()
await ByBit.format_the_dataframe()

(current_daily, next_daily, next_to_next_daily,
current_weekly, next_weekly, next_to_next_weekly,
current_monthly, next_monthly, next_to_next_monthly,
current_quarterly, next_quarterly, next_to_next_quarterly, expiry) = await ByBit.segregate_options_expiry()


In [ ]:
json_fut = await ByBitAPI.get_PerpFutures_Position()
json_fut

In [ ]:
len(json_fut.json()["result"]["list"])

In [ ]:
json_data = await ByBitAPI.get_option_positions()
position_df = await ByBitAPI.format_option_position_dataframe()

In [ ]:
async def set_expiry_delta_and_compute_hedging():
    
    expiry["daily"]["delta_limit"] = float(daily_delta_limit)
    expiry["weekly"]["delta_limit"] = float(weekly_delta_limit)
    expiry["monthly"]["delta_limit"] = float(monthly_delta_limit)
    expiry["quarterly"]["delta_limit"] = float(quarterly_delta_limit)

    position_df.loc[position_df.expiry_type == "daily", "expiry_delta"] = daily_delta_limit
    position_df.loc[position_df.expiry_type == "weekly", "expiry_delta"] = weekly_delta_limit
    position_df.loc[position_df.expiry_type == "monthly", "expiry_delta"] = monthly_delta_limit
    position_df.loc[position_df.expiry_type == "quarterly", "expiry_delta"] = quarterly_delta_limit


    for i in range(len(position_df)):
        print(f"Delta Hedging : {abs(round(position_df.loc[i, "expiry_delta"], 6))}",
              f"Expiry Delta {abs(round(position_df.loc[i, "expiry_delta"], 6))}")
        if abs(round(position_df.loc[i, "delta_hedging"], 6)) > abs(round(position_df.loc[i, "expiry_delta"], 6)):
            print(f"Hedging Required !!! ")
            position_df.loc[i, "hedging_required"] = True
            position_df.loc[i, "PerpFutureQty"] = position_df.loc[i, "delta_hedging"]
        else:
            print(f"Hedging Not Required !!! ")
            position_df.loc[i, "hedging_required"] = False
            position_df.loc[i, "PerpFutureQty"] = position_df.loc[i, "delta_hedging"]
    


In [ ]:
await set_expiry_delta_and_compute_hedging()

In [ ]:
async def check_the_position_availablity():
        
    ## Update the Daily Expiry Dictionary 
    # Update the Daily : Current Daily
    if len(position_df.loc[position_df.expiry == expiry["daily"]["current"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["daily"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["daily"]["current"]["date"], "expiry_type"] = "daily"
        expiry["daily"]["current"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["daily"]["current"]["date"]}")
    
    # Update the Monthly : Next Daily
    if len(position_df.loc[position_df.expiry == expiry["daily"]["next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["daily"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["daily"]["next"]["date"], "expiry_type"] = "daily"
        expiry["daily"]["next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["daily"]["next"]["date"]}")
    
    # Update the Monthly : Next_to_Next Daily
    if len(position_df.loc[position_df.expiry == expiry["daily"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["daily"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["daily"]["next_to_next"]["date"], "expiry_type"] = "daily"
        expiry["daily"]["next_to_next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["daily"]["next_to_next"]["date"]}")
    
    
    
    ## Update the Weekly Expiry Dictionary 
    # Update the Weekly : Current Weekly
    if len(position_df.loc[position_df.expiry == expiry["weekly"]["current"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["weekly"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["weekly"]["current"]["date"], "expiry_type"] = "weekly"
        expiry["weekly"]["current"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["weekly"]["current"]["date"]}")
    
    # Update the Weekly : Next Weekly
    if len(position_df.loc[position_df.expiry == expiry["weekly"]["next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["weekly"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["weekly"]["next"]["date"], "expiry_type"] = "weekly"
        expiry["weekly"]["next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["weekly"]["next"]["date"]}")
    
    # Update the Weekly : Next_to_Next Weekly
    if len(position_df.loc[position_df.expiry == expiry["weekly"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["weekly"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["weekly"]["next_to_next"]["date"], "expiry_type"] = "weekly"
        expiry["weekly"]["next_to_next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["weekly"]["next_to_next"]["date"]}")
    
    ## Update the Monthly Expiry Dictionary 
    # Update the Monthly : Current Monthly
    if len(position_df.loc[position_df.expiry == expiry["monthly"]["current"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["monthly"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["monthly"]["current"]["date"], "expiry_type"] = "monthly"
        expiry["monthly"]["current"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["monthly"]["current"]["date"]}")
    
    # Update the Monthly : Next Monthly
    if len(position_df.loc[position_df.expiry == expiry["monthly"]["next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["monthly"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["monthly"]["next"]["date"], "expiry_type"] = "monthly"
        expiry["monthly"]["next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["monthly"]["next"]["date"]}")
    
    # Update the Monthly : Next_to_Next Monthly
    if len(position_df.loc[position_df.expiry == expiry["monthly"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["monthly"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["monthly"]["next_to_next"]["date"], "expiry_type"] = "monthly"
        expiry["monthly"]["next_to_next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["monthly"]["next_to_next"]["date"]}")
    
    
    
    ## Update the Quarterly Expiry Dictionary 
    # Update the Quarterly : Current Quarterly
    if len(position_df.loc[position_df.expiry == expiry["quarterly"]["current"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["quarterly"]["current"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["quarterly"]["current"]["date"], "expiry_type"] = "quarterly"
        expiry["quarterly"]["current"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["quarterly"]["current"]["date"]}")
    
    # Update the Quarterly : Next Quarterly
    if len(position_df.loc[position_df.expiry == expiry["quarterly"]["next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["quarterly"]["next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["quarterly"]["next"]["date"], "expiry_type"] = "quarterly"
        expiry["quarterly"]["next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["quarterly"]["next"]["date"]}")
    
    # Update the Quarterly : Next_to_Next Quarterly
    if len(position_df.loc[position_df.expiry == expiry["quarterly"]["next_to_next"]["date"], "expiry_type"]) > 0:
        print(f"Position Exist for the Expiry Date : {expiry["quarterly"]["next_to_next"]["date"]}, Updating the Expiry Dictionary")
        position_df.loc[position_df.expiry == expiry["quarterly"]["next_to_next"]["date"], "expiry_type"] = "quarterly"
        expiry["quarterly"]["next_to_next"]["position_exist"] = True
        
    else:
        print(f"No Position Exist for the Expiry Date : {expiry["quarterly"]["next_to_next"]["date"]}")

In [ ]:
await check_the_position_availablity()

In [ ]:
position_df

In [ ]:
current_daily_call, current_daily_put = await recommend_option_position(option_chain=current_daily)
next_daily_call, next_daily_put = await recommend_option_position(option_chain=next_daily)
next_to_next_daily_call, next_to_next_daily_put = await recommend_option_position(option_chain=next_to_next_daily)

current_weekly_call, current_weekly_put = await recommend_option_position(option_chain=current_weekly)
next_weekly_call, next_weekly_put = await recommend_option_position(option_chain=next_weekly)
next_to_next_weekly_call, next_to_next_weekly_put = await recommend_option_position(option_chain=next_to_next_weekly)

current_monthly_call, current_monthly_put = await recommend_option_position(option_chain=current_monthly)
next_monthly_call, next_monthly_put = await recommend_option_position(option_chain=next_monthly)
next_to_next_monthly_call, next_to_next_monthly_put = await recommend_option_position(option_chain=next_to_next_monthly)

current_quarterly_call, current_quarterly_put = await recommend_option_position(option_chain=current_quarterly)
next_quarterly_call, next_quarterly_put = await recommend_option_position(option_chain=next_quarterly)
next_to_next_quarterly_call, next_to_next_quarterly_put = await recommend_option_position(option_chain=next_to_next_quarterly)

In [ ]:
expiry.keys()

In [ ]:
if current_daily_call is not None and current_daily_put is not None and expiry["daily"]["current"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_daily_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_daily_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Daily Option as NO Matching Criteria Exist")

if next_daily_call is not None and next_daily_put is not None and expiry["daily"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_daily_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_daily_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Daily Option as NO Matching Criteria Exist")

if next_to_next_daily_call is not None and next_to_next_daily_put is not None and expiry["daily"]["next_to_next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_daily_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_daily_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Daily Option as NO Matching Criteria Exist")

############
if current_weekly_call is not None and current_weekly_put is not None and expiry["weekly"]["current"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_weekly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_weekly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Weekly Option as NO Matching Criteria Exist")


if next_weekly_call is not None and next_weekly_put is not None and expiry["weekly"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_weekly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_weekly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Weekly Option as NO Matching Criteria Exist")

if next_to_next_weekly_call is not None and next_to_next_weekly_put is not None and expiry["weekly"]["next_to_next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_weekly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_weekly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Weekly Option as NO Matching Criteria Exist")


############
if current_monthly_call is not None and current_monthly_put is not None and expiry["monthly"]["current"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_monthly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_monthly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Monthly Option as NO Matching Criteria Exist")

if next_monthly_call is not None and next_monthly_put is not None and expiry["monthly"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_monthly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_monthly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Monthly Option as NO Matching Criteria Exist")

if next_to_next_monthly_call is not None and next_to_next_monthly_put is not None and expiry["monthly"]["next_to_next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_monthly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_monthly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Monthly Option as NO Matching Criteria Exist")

############
if current_quarterly_call is not None and current_quarterly_put is not None and expiry["quarterly"]["current"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_quarterly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=current_quarterly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Quarterly Option as NO Matching Criteria Exist")

if next_quarterly_call is not None and next_quarterly_put is not None and expiry["quarterly"]["next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_quarterly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_quarterly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next Quarterly Option as NO Matching Criteria Exist")

if next_to_next_quarterly_call is not None and next_to_next_quarterly_put is not None and expiry["quarterly"]["next_to_next"]["position_exist"] is False:
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_quarterly_call.iloc[0]["symbol"],quantity=1.00)
    await ByBitAPI.create_Option_Order(direction="SELL", symbol=next_to_next_quarterly_put.iloc[0]["symbol"],quantity=1.00)
else:
    print(f"Cannot Place Orders for Next to Next Quarterly Option as NO Matching Criteria Exist")


In [ ]:
from ByBit import ByBitAPI

In [ ]:
position_df.info()

In [ ]:
position_df

In [ ]:
expiry_dataframe = position_df.drop(columns=["symbol", "avgPrice", "delta", "theta", "positionValue", "unrealisedPnl", "size", "markPrice",
                            "createdTime", "seq", "updatedTime","side","curRealisedPnl","positionStatus", "gamma",
                            "vega", "option_type","strike_price" ]).copy()

In [ ]:
async def check_delta_computation():
    if round(sum(position_df.total_delta), 6) == round(sum(position_df.delta_hedging)/2, 6):
        print(f"Total Delta is Computed Correctly !!! , Total Options Open Positional Detla is : ",
              f"{sum(position_df.total_delta)}",
              f"Total Delta Computed for Hedging : {sum(position_df.delta_hedging)/2}")
    
    else:
        print(f"ERROR !!!! Delta Computed In-Correctly",
              f"Total Delta : {sum(position_df.total_delta)}",
              f"Total Delta Computed for Hedging : {sum(position_df.delta_hedging)/2}")

In [ ]:
await check_delta_computation()

In [ ]:
expiry_dataframe

In [ ]:
await check_the_position_availablity()

In [ ]:
expiry

In [2]:
from ByBit import CoreLogic

In [3]:
btcCoreLogic = CoreLogic(
    api_url = "https://api-demo.bybit.com",  # API URL
    api_endpoint = "/v5/market/tickers",  # Endpoint URI
    api_keys = "6TzoOeOuyIDiJN9cOu",  # API Keys
    api_secret = "JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",  # API Secret
    baseCoin = "BTC",  # BaseCoin neets be either BTC, ETH, SOL Only
    settleCoin = "USDC",
    default_quantity = 1.00,  # Default Short Quantity

    daily = True,  # Play on Daily Options: True or False
    weekly = True,  # Play on Weekly Options: True or False
    monthly = True,  # Play on Monthly Options: True or False
    quarterly = True,  # Play on Quarterly Options: True or False

    delta_value = 0.07,  # Delta Value beyond which we would take our Positions
    min_bid_price = 0.01,  # Minumum Bid Price to eliminate the Blank Bid Strikes
    initial_mark_price_diff = 0.05,  # The Difference Between Mark & Bid Price 0.05 means 5%
    max_mark_price_diff = 0.3,  # Maximum Difference between Mark & Bid Price
    mark_price_diff_steps = 0.05,  # Mark & Ask Difference Percentage Incremental Step Size
    
    # Tolarated Delta Deavation Limit, after which we Negate the Delta Deavation
    # by hedging with Perpectual Futuers
    daily_delta_limit = 0.10,  # Daily Delta Deviation Limit
    weekly_delta_limit = 0.10,  # Weekly Delta Deviation Limit
    monthly_delta_limit = 0.10,  # Monthly Delta Deviation Limit
    quarterly_delta_limit = 0.10,  # Quarterly Delta Deviation Limit
)

In [4]:
await btcCoreLogic.core_logic_computation()

Class Variables Reinitialization Initiated
Tickers Data Retrieval Success : Data Count : 520
Data Fetching from ByBit API is Completed in : 857.236527997884 ms 

Formatting & Processing of DataFrame is Completed in : 31.50290199846495 ms 

2024-10-30
2024-10-31
2024-11-01
2024-11-08
2024-11-15
2024-11-29
2024-12-27
2025-03-28
2025-06-27
2025-09-26
Initial Current Month is  11
Original Length of Weekly Expiry List Array : 67, But Length of returned Array : 5 
Original Length of Monthly Expiry List Array : 20, But Length of returned Array : 4 
Original Length of Weekly Expiry List Array : 6, But Length of returned Array : 5 
Setting expiry[weekly][current][date] as : 2024-11-01 Setting expiry[weekly][current][exist] as : True
Setting expiry[weekly][next][date] as : 2024-11-08 Setting expiry[weekly][next][exist] as : True
Setting expiry[weekly][next_to_next][date] as : 2024-11-15 Setting expiry[weekly][next_to_next][exist] as : True
Setting expiry[monthly][current][date] as : 2024-11-29 S

In [ ]:
btcCoreLogic.position_df.sort_values("expiry", inplace=True, ignore_index=True)

In [ ]:
import pandas as pd 
btcCoreLogic.position_df.markPrice = round(btcCoreLogic.position_df.markPrice, 6)
btcCoreLogic.position_df.delta = round(btcCoreLogic.position_df.delta, 6)
btcCoreLogic.position_df.theta = round(btcCoreLogic.position_df.theta, 6)
btcCoreLogic.position_df.positionValue = round(btcCoreLogic.position_df.positionValue, 6)

btcCoreLogic.position_df.gamma = round(btcCoreLogic.position_df.gamma, 6)
btcCoreLogic.position_df.vega = round(btcCoreLogic.position_df.vega, 6)
btcCoreLogic.position_df.total_delta = round(btcCoreLogic.position_df.total_delta, 6)
btcCoreLogic.position_df.expiry_delta = round(btcCoreLogic.position_df.expiry_delta, 6)
btcCoreLogic.position_df.delta_hedging = round(btcCoreLogic.position_df.delta_hedging, 6)
btcCoreLogic.position_df.PerpFutureQty = round(btcCoreLogic.position_df.PerpFutureQty, 6)
# btcCoreLogic.position_df['expiry'] = pd.to_datetime(btcCoreLogic.position_df['expiry'], format='%y%m%d')

In [7]:
btcCoreLogic.position_df

,symbol,avgPrice,delta,theta,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,...,vega,option_type,strike_price,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,BTC-28MAR25-40000-P,420.0,0.032844,6.813937,-413.460904,6.539096,413.460904,2024-10-29 08:28:58.827,671343898,2024-10-29 08:28:58.827,...,-34.945382,PUT,40000.0,2025-03-28,0.032844,0.1,quarterly,-0.058393,False,-0.058393
1,BTC-28MAR25-200000-C,257.5,-0.049745,13.045619,-545.721910,-30.721910,272.860955,2024-10-18 15:47:30.703,671343897,2024-10-29 08:28:58.639,...,-55.462895,CALL,200000.0,2025-03-28,-0.099489,0.1,quarterly,-0.058393,False,-0.058393
2,BTC-27DEC24-52000-P,490.0,0.061837,17.457988,-495.919573,-5.919573,495.919573,2024-10-29 08:28:58.350,671343896,2024-10-29 08:28:58.351,...,-35.519038,PUT,52000.0,2024-12-27,0.061837,0.1,monthly,-0.006449,False,-0.006449
3,BTC-27DEC24-110000-C,430.0,-0.068286,20.884123,-510.471433,-80.471433,510.471433,2024-10-29 08:28:58.025,671343895,2024-10-29 08:28:58.025,...,-38.363900,CALL,110000.0,2024-12-27,-0.068286,0.1,monthly,-0.006449,False,-0.006449
4,BTC-29NOV24-56000-P,385.0,0.066092,26.137245,-391.725312,-6.725312,391.725312,2024-10-29 08:28:57.747,671343894,2024-10-29 08:28:57.747,...,-26.875745,PUT,56000.0,2024-11-29,0.066092,0.1,monthly,-0.193835,True,-0.193835
5,BTC-29NOV24-98000-C,300.0,-0.129964,56.703921,-734.060583,-134.060583,367.030291,2024-10-18 06:00:38.346,671343893,2024-10-29 08:28:57.562,...,-53.049869,CALL,98000.0,2024-11-29,-0.259927,0.1,monthly,-0.193835,True,-0.193835
6,BTC-8NOV24-60000-P,245.0,0.062687,51.849533,-238.492128,6.507873,238.492128,2024-10-29 08:28:56.909,671343891,2024-10-29 08:28:56.909,...,-14.543746,PUT,60000.0,2024-11-08,0.062687,0.1,weekly,-0.135189,True,-0.135189
7,BTC-8NOV24-88000-C,180.0,-0.098938,89.586180,-344.970771,15.029229,172.485385,2024-10-18 06:00:37.732,671343890,2024-10-29 08:28:56.664,...,-24.129881,CALL,88000.0,2024-11-08,-0.197876,0.1,weekly,-0.135189,True,-0.135189
8,BTC-31OCT24-67500-P,70.0,0.061173,75.327900,-66.155914,3.844086,66.155914,2024-10-29 08:28:56.204,671343889,2024-10-29 08:28:56.204,...,-6.335974,PUT,67500.0,2024-10-31,0.061173,0.1,daily,-0.129325,True,-0.129325
9,BTC-31OCT24-76000-C,60.0,-0.110851,164.049201,-134.736644,-14.736644,67.368322,2024-10-29 03:37:35.204,671343888,2024-10-29 08:28:55.876,...,-11.725850,CALL,76000.0,2024-10-31,-0.221703,0.1,daily,-0.129325,True,-0.129325


In [ ]:
btcCoreLogic.position_df

In [ ]:
expiry_df = btcCoreLogic.position_df.drop(columns=["symbol", "avgPrice", "delta", "theta", "positionValue", "unrealisedPnl", "size", "markPrice",
                            "createdTime", "seq", "updatedTime","side","curRealisedPnl","positionStatus", "gamma",
                            "vega", "option_type","strike_price" ]).copy()

In [ ]:
expiry_df.drop_duplicates("expiry", inplace=True, ignore_index=True)

In [ ]:
expiry_df

In [ ]:
btcCoreLogic.position_df.loc[btcCoreLogic.position_df.expiry == expiry_df.loc[0,"expiry"]]

In [ ]:
import numpy as np 
avgPrice = np.array([-5000.00 for _ in range(len(expiry_df))])
markPrice = np.array([-5000.00 for _ in range(len(expiry_df))])
unrealisedPnl = np.array([-5000.00 for _ in range(len(expiry_df))])

expiry_df["avgPrice"] = avgPrice
expiry_df["markPrice"] = markPrice
expiry_df["unrealisedPnl"] = unrealisedPnl


In [ ]:
for i in range(len(expiry_df)):
    expiry_df.loc[i, "markPrice"] = sum(btcCoreLogic.position_df.loc[
                                        btcCoreLogic.position_df.expiry == expiry_df.loc[i , "expiry"], "markPrice"])
    expiry_df.loc[i, "avgPrice"] = sum(btcCoreLogic.position_df.loc[
                                       btcCoreLogic.position_df.expiry == expiry_df.loc[i ,"expiry"], "avgPrice"])
    expiry_df.loc[i, "unrealisedPnl"] = sum(btcCoreLogic.position_df.loc[
                                            btcCoreLogic.position_df.expiry == expiry_df.loc[i , "expiry"], "unrealisedPnl"])
    

In [ ]:
expiry_df

In [ ]:
delta_risk_multiplier:float = 2.0
total_perpFutures:float = 0.0
temp_count:float = 0.0
total_delta_risk_magnitude:float = 0.0


total_perpFutures = round(sum(expiry_df.loc[expiry_df.hedging_required == True, "PerpFutureQty"]), 6)
total_perpFutures

In [ ]:
daily_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "daily"].PerpFutureQty), 6)
daily_delta_risk_magnitude = delta_risk_multiplier * 0.1

if abs(daily_delta_risk) > abs(daily_delta_risk_magnitude):
    
    total_delta_risk_magnitude = total_delta_risk_magnitude + daily_delta_risk
    
    print(f"Attention !!!! : Daily Delta Risk Paramater Exceeded : {daily_delta_risk_magnitude}, "
    f"Risk Magnitude : {daily_delta_risk}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Daily Delta Risk Paramater is within Limit : {daily_delta_risk_magnitude} ",
         f"Risk Magnitude : {daily_delta_risk}, Total Risk Magnitude: {total_delta_risk_magnitude}")

In [ ]:
expiry_df[expiry_df.expiry_type == "weekly"]

In [ ]:
weekly_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "weekly"].PerpFutureQty), 6)
weekly_delta_risk_magnitude = delta_risk_multiplier * 0.1

if abs(weekly_delta_risk) > abs(weekly_delta_risk_magnitude):
    
    total_delta_risk_magnitude = total_delta_risk_magnitude + weekly_delta_risk

    print(f"Attention !!!! : Weekly Delta Risk Paramater Exceeded : {weekly_delta_risk}, "
    f"Risk Magnitude : {weekly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Weekly Delta Risk Paramater is within Limit : {weekly_delta_risk} ",
         f"Risk Magnitude : {weekly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

In [ ]:
expiry_df[expiry_df.expiry_type == "monthly"]

In [ ]:
monthly_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "monthly"].PerpFutureQty), 6)
monthly_delta_risk_magnitude = delta_risk_multiplier * 0.1

if abs(monthly_delta_risk) > abs(monthly_delta_risk_magnitude):
    total_delta_risk_magnitude = total_delta_risk_magnitude + monthly_delta_risk

    print(f"Attention !!!! : Monthly Delta Risk Paramater Exceeded : {monthly_delta_risk}, "
    f"Risk Magnitude : {monthly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Monthly Delta Risk Paramater is within Limit : {monthly_delta_risk} ",
         f"Risk Magnitude : {monthly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

In [ ]:
expiry_df[expiry_df.expiry_type == "quarterly"]

In [ ]:
quarterly_delta_risk = round(sum(expiry_df[expiry_df.expiry_type == "quarterly"].PerpFutureQty), 6)
quarterly_delta_risk_magnitude = delta_risk_multiplier * 0.1

if quarterly_delta_risk > quarterly_delta_risk_magnitude:
    
    total_delta_risk_magnitude = total_delta_risk_magnitude + quarterly_delta_risk
    
    print(f"Attention !!!! : Quarterly Delta Risk Paramater Exceeded : {quarterly_delta_risk}, "
    f"Risk Magnitude : {quarterly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

else:
    print(f"Quarterly Delta Risk Paramater is within Limit : {quarterly_delta_risk} ",
         f"Risk Magnitude : {quarterly_delta_risk_magnitude}, Total Risk Magnitude: {total_delta_risk_magnitude}")

In [ ]:
sum((btcCoreLogic.position_df["size"] * btcCoreLogic.position_df["avgPrice"]) - (btcCoreLogic.position_df["markPrice"] * btcCoreLogic.position_df["size"]))


In [ ]:
sum((btcCoreLogic.position_df["size"] * btcCoreLogic.position_df["avgPrice"]) - (btcCoreLogic.position_df["markPrice"] * btcCoreLogic.position_df["size"]))


In [ ]:
((total_perpFutures >= total_delta_risk_magnitude) and total_perpFutures != 0.0), total_perpFutures >= total_delta_risk_magnitude 


In [ ]:
total_delta_risk_magnitude, total_perpFutures, 


In [8]:
from ByBit import ByBitAPI
baseCoin = "BTC"
ByBitAPI = ByBitAPI(default_quantity = 1.0,
                    api_url = "https://api-demo.bybit.com",
                    api_key = "6TzoOeOuyIDiJN9cOu",
                    api_secret = "JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",
                    baseCoin = "BTC",                 
                    settleCoin = "USDC")

In [23]:
try :
    PerpFutures_json_data = await ByBitAPI.get_PerpFutures_Position()
    if len(PerpFutures_json_data.json()["result"]["list"]) > 0:
        PerpFutures_df = await ByBitAPI.format_perpfutures()
        
        if (PerpFutures_df.loc[0, "side"] == "BUY") or (PerpFutures_df.loc[0, "side"] == "Buy"):
            PerpFutures_position = float(PerpFutures_df.loc[0,"size"])
            PerpFuturePnL = (PerpFutures_df.loc[0, "markPrice"] - PerpFutures_df.loc[0, "avgPrice"]) * PerpFutures_df.loc[0, "size"]

        elif (PerpFutures_df.loc[0, "side"] == "SELL") or (PerpFutures_df.loc[0, "side"] == "Sell"):
            PerpFutures_position = float(PerpFutures_df.loc[0,"size"]) * (-1.0)
            PerpFuturePnL = (PerpFutures_df.loc[0, "avgPrice"] - PerpFutures_df.loc[0, "markPrice"]) * PerpFutures_df.loc[0, "size"]

        print(f"Existing Perpetual Futures Position Size : {PerpFutures_position} , PnL : ${round(PerpFuturePnL, 3)}")
        
    else:
        PerpFutures_position = 0.0
        print(f"No {baseCoin}, Perpectual Futires position Exists, Setting Value for 'PerpFutures_position'  :  {PerpFutures_position}")
except Exception as e:
    print(f"Error in Fetcting the Perpectual Futures Position Error Details : {e}")

Parameter String is : 17302649721426TzoOeOuyIDiJN9cOu5000category=linear&baseCoin=BTC&settleCoin=USDC

Generated Signature is : ce404bd2348b88fa940bc58d5957cdb66208f2316f90526fec55bde651818850

Perpectual Futures Position Info API Signature : ce404bd2348b88fa940bc58d5957cdb66208f2316f90526fec55bde651818850

API Header is : {'X-BAPI-API-KEY': '6TzoOeOuyIDiJN9cOu', 'X-BAPI-SIGN': 'ce404bd2348b88fa940bc58d5957cdb66208f2316f90526fec55bde651818850', 'X-BAPI-SIGN-TYPE': '2', 'X-BAPI-TIMESTAMP': '1730264972142', 'X-BAPI-RECV-WINDOW': '5000', 'Content-Type': 'application/json'} 

Final URL is : https://api-demo.bybit.com/v5/position/list?category=linear&baseCoin=BTC&settleCoin=USDC
Positions Info Length is : 1
Existing Perpetual Futures Position Size : 0.546 , PnL : $698.825


In [25]:
await ByBitAPI.get_option_positions()
tempdf = await ByBitAPI.format_option_position_dataframe()

Parameter String is : 17302651591316TzoOeOuyIDiJN9cOu5000category=option&baseCoin=BTC&settleCoin=USDC

Generated Signature is : 4443d1bfccf434f976f63f62e6fcf81af37df0ed8458224c34797a4e85331582

Option Position Info API Signature : 4443d1bfccf434f976f63f62e6fcf81af37df0ed8458224c34797a4e85331582

API Header is : {'X-BAPI-API-KEY': '6TzoOeOuyIDiJN9cOu', 'X-BAPI-SIGN': '4443d1bfccf434f976f63f62e6fcf81af37df0ed8458224c34797a4e85331582', 'X-BAPI-SIGN-TYPE': '2', 'X-BAPI-TIMESTAMP': '1730265159131', 'X-BAPI-RECV-WINDOW': '5000', 'Content-Type': 'application/json'} 

Final URL is : https://api-demo.bybit.com/v5/position/list?category=option&baseCoin=BTC&settleCoin=USDC
Positions Info Length is : 20
For Index Position 0, Symbol Data was Split Correctly : ['BTC', '1NOV24', '66500', 'P'] Expiry Data: 2024-11-01 Strike Price: 66500.0 Options Type Data: PUT,Total Delta : 0.1069404 ,Delta : 0.05347020 

For Index Position 1, Symbol Data was Split Correctly : ['BTC', '1NOV24', '78000', 'C'] Expiry D

In [28]:
sum(tempdf.unrealisedPnl)

-500.88511872

In [ ]:
# Compute the Perpectual Futures Position Requirements

if abs(total_perpFutures) >= abs(total_delta_risk_magnitude):
    PerpFut_Requirement = total_delta_risk_magnitude
elif abs(total_perpFutures) < abs(total_delta_risk_magnitude):
    PerpFut_Requirement = 0.0


In [ ]:
PerpFutures_position, total_perpFutures, PerpFut_Requirement

In [ ]:
if PerpFut_Requirement == 0:
    print(f"No Perpectual Futures Position is Required as Requirements is : {PerpFut_Requirement}")
elif PerpFut_Requirement > 0:
    PerpFut_Requirement = round((PerpFut_Requirement * (-1.0)), 3)
    print(f"We need to Go Short PerpFuture with following Quantity : {abs(PerpFut_Requirement)}, "
          f"Setting PerpFuture Quantity as : {PerpFut_Requirement} ")
elif PerpFut_Requirement < 0:
    PerpFut_Requirement = round((PerpFut_Requirement * (-1.0)) , 3)
    print(f"We need to Go Long PerpFuture with following Quantity : {abs(PerpFut_Requirement)}, "
          f"Setting PerpFuture Quantity as : {PerpFut_Requirement} ")

In [ ]:
def check_PerpFutures_Quantity(PerpFut_Position, PerpFut_Requirement):
    
    adjustment_quantity = 0.0
    if PerpFut_Position == 0.0: 
        
        print(f"No PerpFuture Position Exist, PerpFuture Position {PerpFut_Position}, Will Process the PerpFuture Requirements as is")
        
        if PerpFut_Requirement == 0:
            print(f"No PerpFuture Position is Required : {PerpFut_Requirement}, No Order Processing is Required")
            
        elif PerpFut_Requirement > 0:
            print(f"Placing a 'BUY' Order for PerpFuture with following Quantity : {PerpFut_Requirement}")
            
        elif PerpFut_Requirement < 0:
            print(f"Placing a 'SELL' Order for PerpFuture with following Quantity : {PerpFut_Requirement}")
            
    elif PerpFut_Position > 0.0:
        
        if PerpFut_Requirement == 0:
            adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)
            
            print(f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                  f"So Futures Adjustment 'SELL' Quantity is : {adjustment_quantity}")
            
        elif PerpFut_Requirement > 0:
            
            if PerpFut_Position > PerpFut_Requirement:
                adjustment_quantity = -round((PerpFut_Position - PerpFut_Requirement), 3)
            elif PerpFut_Position < PerpFut_Requirement:
                adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)
                
            if adjustment_quantity < 0:
                print(f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                      f"So Futures Adjustment 'SELL' Quantity is : {adjustment_quantity}")
            else:
                print(f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                      f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")
            
        elif PerpFut_Requirement < 0:
            adjustment_quantity = round((-PerpFut_Position + PerpFut_Requirement), 3)
    
            if adjustment_quantity < 0:
                print(f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                      f"So Futures Adjustment 'SELL' Quantity is : {adjustment_quantity}")
            else:
                print(f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                      f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")
        
    
    elif PerpFut_Position < 0.0: 
        
        if PerpFut_Requirement == 0:
            adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)
            
            print(f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                  f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")
            
        elif PerpFut_Requirement > 0:
            adjustment_quantity = round((-PerpFut_Position + PerpFut_Requirement), 3)
            
            assert adjustment_quantity > 0.0
            
            print(f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                  f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")
    
            
        elif PerpFut_Requirement < 0:
            if PerpFut_Position > PerpFut_Requirement:
                adjustment_quantity = -round((PerpFut_Position - PerpFut_Requirement), 3)
            elif PerpFut_Position < PerpFut_Requirement:
                adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)
    
            if adjustment_quantity < 0:
                print(f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                      f"So Futures Adjustment 'SELL' Quantity is : {adjustment_quantity}")
            else:
                print(f"Since Existing PerpFuture Position Exist : {PerpFut_Position}, and PerpFuture Requirements : {PerpFut_Requirement},"
                      f"So Futures Adjustment 'BUY' Quantity is : {adjustment_quantity}")
    
    return adjustment_quantity


In [ ]:
check_PerpFutures_Quantity(PerpFut_Position = PerpFutures_position, PerpFut_Requirement=PerpFut_Requirement)

In [ ]:
PerpFut_Position = 0.02
PerpFut_Requirement = 0.01
if PerpFut_Position > PerpFut_Requirement:
    adjustment_quantity = -round((PerpFut_Position - PerpFut_Requirement), 3)
elif PerpFut_Position < PerpFut_Requirement:
    adjustment_quantity = round((PerpFut_Requirement - PerpFut_Position), 3)

adjustment_quantity

In [ ]:
unrealisedPnL = round(sum(expiry_df[expiry_df["expiry_type"] == "daily"].unrealisedPnl), 6)
print(f"Daily unrealised PnL: {unrealisedPnL} ")

unrealisedPnL = round(sum(expiry_df[expiry_df["expiry_type"] == "weekly"].unrealisedPnl), 6)
print(f"Weekly unrealised PnL: {unrealisedPnL} ")

unrealisedPnL = round(sum(expiry_df[expiry_df["expiry_type"] == "monthly"].unrealisedPnl), 6)
print(f"Monthly unrealised PnL: {unrealisedPnL} ")

unrealisedPnL = round(sum(expiry_df[expiry_df["expiry_type"] == "quarterly"].unrealisedPnl), 6)
print(f"Quarterly unrealised PnL: {unrealisedPnL} ")


In [ ]:
sum(btcCoreLogic.position_df[btcCoreLogic.position_df["expiry_type"] == "monthly"].unrealisedPnl)

In [ ]:
check_PerpFutures_Quantity(0.0, 0.0)

In [ ]:
check_PerpFutures_Quantity(0.0, 0.1)

In [ ]:
check_PerpFutures_Quantity(0.0, -0.1)

In [ ]:
check_PerpFutures_Quantity(0.1, 0.0)

In [ ]:
check_PerpFutures_Quantity(0.1, 0.2)

In [ ]:
check_PerpFutures_Quantity(0.1, 0.1)

In [ ]:
check_PerpFutures_Quantity(0.2, 0.1)

In [ ]:
check_PerpFutures_Quantity(-0.1, 0.0)

In [ ]:
check_PerpFutures_Quantity(-0.1, 0.1)

In [ ]:
check_PerpFutures_Quantity(-0.1, -0.1)

In [ ]:
check_PerpFutures_Quantity(-0.2, -0.1)

In [ ]:
check_PerpFutures_Quantity(-0.2, -0.1)

In [ ]:
time.sleep(10)

In [ ]:
await asyncio.sleep(5)

In [ ]:
import datetime 
import time
import time
import asyncio

In [ ]:
utcnow = datetime.datetime.now(datetime.UTC).
utcnow

In [ ]:
if utcnow.minute % 5 == 0 and utcnow.second == 0:
    print(executing Function Current time {str(utcnow)})

else:
    await asyncio.sleep( )

In [ ]:
str(utcnow), (5 - (datetime.datetime.now(datetime.UTC).minute % 5)) * 60, datetime.datetime.now(datetime.UTC).second 


In [ ]:
(5 - (datetime.datetime.now(datetime.UTC).minute % 5)) * 60 + datetime.datetime.now(datetime.UTC).second 


In [ ]:
i = 0
while i <= 3:
    current_utc_time = datetime.datetime.now(datetime.UTC)
    if (current_utc_time.minute % 5 ) == 0:
        print(f"Executimg Function {str(current_utc_time)},"
              f"Wait Timer : {(4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second)}")
        await asyncio.sleep(60)
    else:
        print(f"Entering Sleep at {str(current_utc_time)}",
             f"Wait Timer : {(4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second)}")
        await asyncio.sleep((4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second))
    i = i + 1

In [ ]:
current_utc_time = datetime.datetime.now(datetime.UTC)

(4 - (current_utc_time.minute % 5)) * 60 + (60 - current_utc_time.second), current_utc_time.minute, current_utc_time.second

In [ ]:
current_utc_time

In [ ]:
expiry_df

In [ ]:

for i in range(len(expiry_df)):
    if abs(expiry_df.loc[i, "delta_hedging"]) > abs(expiry_df.loc[i, "expiry_delta"] * 2 ):
        print(f"Expiry : {expiry_df.loc[i, "expiry"]},"
              f"Exceeded the Risk Tolerance Limit : { (abs(expiry_df.loc[i, "expiry_delta"] * 2 ))}",
              f"Current Delta Risk is {expiry_df.loc[i, "delta_hedging"]}")

        print(f"{btcCoreLogic.position_df[btcCoreLogic.position_df.expiry == str(expiry_df.loc[i, "expiry"])]["symbol"]}")

In [ ]:
btcCoreLogic.position_df[btcCoreLogic.position_df.expiry == str(expiry_df.loc[0, "expiry"])]

In [ ]:
btcCoreLogic.position_df.info()

In [ ]:
expiry_df.info()

In [ ]:
import pandas as pd
btcCoreLogic.position_df['expiry'] = pd.to_datetime(btcCoreLogic.position_df['expiry'], format='%y%m%d')

In [ ]:
btcCoreLogic.position_df.expiry == "2024-10-29"